In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import json
import plotly.graph_objects as go
from datetime import datetime
import folium
from folium import plugins
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

In [ ]:
df=pd.read_csv(r'shopping_trends.csv')
df

In [ ]:
df.info()

In [ ]:
df['Purchase Amount (USD)']=df['Purchase Amount (USD)'].astype(float)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df_gender_count = df.groupby('Gender').size().reset_index(name='Count')
df_gender_count

In [ ]:
fig_gender_count_pie = px.pie(df_gender_count, 
                        values = 'Count', 
                        names = 'Gender',
                        title = 'Total Number of Orders by Gender',
                        color='Gender',
                        hole=0.3, height = 300, width = 600,
                              color_discrete_map={
                                 'Male': 'dodgerblue',
                                 'Female': 'orangered'
                             })
fig_gender_count_pie.update_traces(textposition='inside',
                             textinfo='label+value+percent')
fig_gender_count_pie.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    showlegend=False,
    margin=dict(t=50, b=30),
    
)
fig_gender_count_pie

In [ ]:
df_gender_purchase_amount = df.pivot_table(index='Gender', values='Purchase Amount (USD)', aggfunc='sum').reset_index()
df_gender_purchase_amount['Purchase Amount (in Thousand Dollars)'] = df_gender_purchase_amount['Purchase Amount (USD)'].apply(lambda x: f"{x/1_000:.2f}K")
df_gender_purchase_amount = df_gender_purchase_amount.sort_values(by='Purchase Amount (USD)', ascending=False).reset_index(drop=True)

print(df_gender_purchase_amount)

In [ ]:
fig_gender_amount_bar = px.bar(df_gender_purchase_amount,
                               x='Gender', 
                               y='Purchase Amount (USD)',
                               title='Total Purchase Amount by Gender (in Thousand Dollars)',
                               text='Purchase Amount (in Thousand Dollars)', 
                               height=350, width=600,
                               color='Gender',
                               color_discrete_map={
                                 'Male': 'dodgerblue',
                                 'Female': 'orangered'}, 
                              )

fig_gender_amount_bar.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    showlegend=False,
    margin = dict(b=30)
    
)
fig_gender_amount_bar

In [ ]:
bins = [18, 24, 34, 44, 54, 64, 71]
labels = ['18-24', '25-34', '35-44', '45-54', '55-64','65-70']
df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)

In [ ]:
df_age_group_order_bin = df.groupby(['Age Group','Gender'], observed=True).size().reset_index(name='Count')
total_per_age_group = df_age_group_order_bin.groupby('Age Group', observed=True)['Count'].sum().reset_index()
print(df_age_group_order_bin)
print(total_per_age_group)

In [ ]:
fig_age_group_bin_order_bar = px.bar(df_age_group_order_bin, 
                                     x='Age Group', 
                                     y='Count', 
                                     title='Number of Orders by Age Group and Gender', 
                                     color = 'Gender',
                                     text = 'Count',
                                     barmode='stack', 
                                     color_discrete_map={
                                         'Male': 'dodgerblue',
                                         'Female': 'orangered'}, height=300, width=600 )
# Add annotations for the totals
for i, row in total_per_age_group.iterrows():
    fig_age_group_bin_order_bar.add_annotation(
        x=row['Age Group'],
        y=row['Count'],
        text=str(row['Count']),
        showarrow=False,
        font=dict(size=12, color='black'),
        xanchor='center',
        yanchor='bottom'
    )

fig_age_group_bin_order_bar.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    barmode='stack',
    yaxis={'showgrid':True, 'gridcolor':'LightGray'}
)

fig_age_group_bin_order_bar

In [ ]:
df_category_orders = df.groupby(['Category','Gender'], observed=True).size().reset_index(name='count')
df_category_orders['Count'] = df_category_orders['count'].apply(lambda x: f"{x:,}")
df_category_orders = df_category_orders.sort_values(by='count', ascending=True).reset_index(drop=True)

total_category_orders = df_category_orders.groupby('Category', observed=True)['count'].sum().reset_index()
total_category_orders['Count'] = total_category_orders['count'].apply(lambda x: f"{x:,}")
total_category_orders = total_category_orders.sort_values(by='count', ascending=True).reset_index(drop=True)

print(df_category_orders)
print(total_category_orders)

In [ ]:
fig_category_orders = px.bar(df_category_orders, 
                             x='count', 
                             y='Category', 
                             title = 'Order Count by Category and Gender',
                             text='count', 
                             color='Gender', 
                             orientation='h',
                             height=300,
                             width=700,
                             barmode='stack', 
                             color_discrete_map={
                                 'Male': 'dodgerblue',
                                 'Female': 'orangered'
                             })



# Add annotations for the 
for i, row in total_category_orders.iterrows():
    fig_category_orders.add_annotation(
        x=row['count'],
        y=row['Category'],
        text=str(row['Count']),
        showarrow=False,
        font=dict(size=12, color='black'),
        xanchor='left',
        yanchor='middle'
    )

fig_category_orders.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    xaxis={'showgrid':True, 'gridcolor':'LightGray'}
)


fig_category_orders

In [ ]:
df_category_purchase_amount = df.pivot_table(index=['Category'], values='Purchase Amount (USD)', aggfunc = 'sum').reset_index()
df_category_purchase_amount['Purchase Amount (in Thousand Dollars)'] = df_category_purchase_amount['Purchase Amount (USD)'].apply(lambda x: f"{x/1_000}K")
df_category_purchase_amount = df_category_purchase_amount.sort_values(by='Purchase Amount (USD)', ascending=True).reset_index(drop=True)

total_category_purchase_amount = df_category_purchase_amount.groupby(['Category'], observed=True)['Purchase Amount (USD)'].sum().reset_index()
total_category_purchase_amount['Purchase Amount (in Thousand Dollars)'] = total_category_purchase_amount['Purchase Amount (USD)'].apply(lambda x: f"{x / 1_000:.2f}K")
total_category_purchase_amount = total_category_purchase_amount.sort_values(by='Purchase Amount (USD)', ascending=True).reset_index(drop=True)

print(df_category_purchase_amount)
print(total_category_purchase_amount)

In [ ]:
fig_category_purchase_amount = px.bar(df_category_purchase_amount, 
                                      x='Purchase Amount (USD)', 
                                      y='Category', 
                                      title = 'Total Purchase Amount by Category (in Thousand Dollars)',
                                      text='Purchase Amount (in Thousand Dollars)', 
                                      orientation='h', 
                                      height=300,
                                      width=700)

fig_category_purchase_amount.update_traces(
    texttemplate='%{text}', 
    textposition='inside',
    textangle=0,
    textfont_size=10
)


fig_category_purchase_amount.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    xaxis={'showgrid':True, 'gridcolor':'LightGray'}
)

fig_category_purchase_amount

In [ ]:
df_item_purchased_count = df.groupby(['Item Purchased','Gender'], observed=True).size().reset_index(name='Count')
df_item_purchased_count = df_item_purchased_count.sort_values(by=['Count'], ascending=True).reset_index(drop=True)

total_counts = df_item_purchased_count.groupby('Item Purchased')['Count'].sum().reset_index()
total_counts = total_counts.sort_values(by='Count', ascending=True).reset_index(drop=True)

# Merge the sorted total counts back to the original DataFrame
df_item_purchased_count = df_item_purchased_count.merge(total_counts, on='Item Purchased', suffixes=('', '_Total'))

# Sort the DataFrame by the total count to maintain the order in the bar chart
df_item_purchased_count = df_item_purchased_count.sort_values(by='Count_Total', ascending=True).reset_index(drop=True)


print(total_counts)
print(df_item_purchased_count)

In [ ]:
fig_item_purchased_count = px.bar(df_item_purchased_count, 
                                  x='Count', y='Item Purchased',  
                                  title='Order Count by Items and Gender',
                                  orientation='h',
                                  text='Count',
                                  height=600, width=450,
                                  barmode='stack', 
                                  color='Gender',
                                  color_discrete_map={'Male': 'dodgerblue','Female': 'orangered'}
)

for i, row in total_counts.iterrows():
    fig_item_purchased_count.add_annotation(
        x=row['Count'],
        y=row['Item Purchased'],
        text=str(row['Count']),
        showarrow=False,
        font=dict(size=12, color='black'),
        xanchor='left',
        yanchor='middle'
    )

fig_item_purchased_count.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    xaxis={'showgrid':True, 'gridcolor':'LightGray'},
    legend=dict(
        orientation='h',  # Horizontal
        yanchor='bottom', 
        y=-0.2,  # Position below the plot
        xanchor='center',
        x=0.5  # Centered horizontally
    ),
    margin=dict(t=40, b=10, r=20)
)
fig_item_purchased_count

In [ ]:
df_item_purchased_amount=df.pivot_table(index='Item Purchased', values='Purchase Amount (USD)', aggfunc='sum').reset_index()
df_item_purchased_amount = df_item_purchased_amount.sort_values(by='Purchase Amount (USD)', ascending=True).reset_index(drop=True)
df_item_purchased_amount['Purchase Amount (in Thousand Dollars)'] = df_item_purchased_amount['Purchase Amount (USD)'].apply(lambda x: f"{x/1_000:.2f}K")


print(df_item_purchased_amount)

In [ ]:
fig_item_purchase_amount = px.bar(df_item_purchased_amount, 
                                   x='Purchase Amount (USD)', 
                                   y='Item Purchased', 
                                   title='Total Amount Purchased by Item',
                                   orientation='h', 
                                   text='Purchase Amount (in Thousand Dollars)',
                                   height=600, width=450)

fig_item_purchase_amount.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    xaxis={'showgrid':True, 'gridcolor':'LightGray'},
    margin=dict(t=40,b=30, r=20)
)

fig_item_purchase_amount

In [ ]:
df_location = df.pivot_table(index='Location', values=['Customer ID','Purchase Amount (USD)'], aggfunc={'Customer ID':'count','Purchase Amount (USD)':'sum'}).reset_index()
df_location['Purchase Amount (in Thousand Dollars)']=df_location['Purchase Amount (USD)'].apply(lambda x: F"{x/1_000:.2f}K")
df_location.columns=['Location','Order Count','Purchase Amount (USD)','Purchase Amount (in Thousand Dollars)']
df_location['Latitude']=[32.806671, 61.370716, 33.729759, 34.969704, 36.116203, 39.059811, 41.597782, 39.318523, 27.766279, 33.040619, 21.094318, 44.240459, 40.349457, 39.849426, 42.011539, 38.5266, 37.66814, 31.169546, 44.693947, 39.063946, 42.230171, 43.326618, 45.694454, 32.741646, 38.456085, 46.921925, 41.12537, 38.313515, 43.452492, 40.298904, 34.840515, 42.165726, 35.630066, 47.528912, 40.388783, 35.565342, 44.572021, 40.590752, 41.680893, 33.856892, 44.299782, 35.747845, 31.054487, 40.150032, 44.045876, 37.769337, 47.400902, 38.491226, 44.268543, 42.755966]
df_location['Longitude']=[-86.79113, -152.404419, -111.431221, -92.373123, -119.681564, -105.311104, -72.755371, -75.507141, -81.686783, -83.643074, -157.498337, -114.478828, -88.986137, -86.258278, -93.210526, -96.726486, -84.670067, -91.867805, -69.381927, -76.802101, -71.530106, -84.536095, -93.900192, -89.678696, -92.288368, -110.454353, -98.268082, -117.055374, -71.563896, -74.521011, -106.248482, -74.948051, -79.806419, -99.784012, -82.764915, -96.928917, -122.070938, -77.209755, -71.51178, -80.945007, -99.438828, -86.692345, -97.563461, -111.862434, -72.710686, -78.169968, -121.490494, -80.954456, -89.616508, -107.30249]
df_location

In [ ]:


# Create the base map
map_location = folium.Map(location=[37.0902, -95.7129], zoom_start=5)

# Create a Choropleth layer
folium.Choropleth(
    geo_data='us-states.json',  # Path to GeoJSON file
    name='choropleth',
    data=df_location,
    columns=['Location', 'Order Count'],
    key_on='feature.properties.name',
    fill_color='Blues',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='ORder Count'
).add_to(map_location)

# Save the map to an HTML file
map_location.save("choropleth_map.html")


In [ ]:
df_location_sorted_order = df_location.sort_values(by=['Order Count'], ascending=False).reset_index(drop=True)
df_location_sorted_order

In [ ]:
fig_order_per_location = px.bar(df_location_sorted_order, x='Location', y='Order Count',
                                title='Order Count by State', text='Order Count', height=400, width=1200,)

fig_order_per_location.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    yaxis={'showgrid':True, 'gridcolor':'LightGray'},
    margin=dict(t=40, b=10, r=20)
)

fig_order_per_location

In [ ]:
df_location_sorted_amount = df_location.sort_values(by=['Purchase Amount (USD)'], ascending=False).reset_index(drop=True)
df_location_sorted_amount

In [ ]:
fig_order_per_location_amount = px.bar(df_location_sorted_amount, x='Location', y='Purchase Amount (USD)',
                                       title='Total Purchase Amount by State (in Thousand Dollars)', 
                                       text='Purchase Amount (in Thousand Dollars)', height=400, width=1200,)

fig_order_per_location_amount.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    yaxis={'showgrid':True, 'gridcolor':'LightGray'},
    margin=dict(t=40, b=10, r=20)
)

fig_order_per_location_amount

In [ ]:
df_sizes = df.groupby('Size', observed=True).size().reset_index(name='Count')
df_sizes

In [ ]:
fig_pie_sizes = px.pie(df_sizes, values='Count', names='Size', title = 'Order Distribution by Sizes',
                        hole=0.3, height = 350, width = 400)
fig_pie_sizes.update_traces(textposition='inside',
                             textinfo='label+value+percent')
fig_pie_sizes.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    showlegend=False,
    margin=dict(t=50, b=30),
    
)
fig_pie_sizes

In [ ]:
df_color = df.groupby('Color', observed=True).size().reset_index(name='Count')
df_color = df_color.sort_values(by='Count', ascending=False).reset_index(drop=True)
df_color_top5 = df_color.head(5)
print(df_color_top5)

In [ ]:
fig_color = px.bar(df_color_top5, x='Color', y='Count', text='Count', height=350, width=400,
                   title='Top 5 Most Popular Colors',  color='Color',
                                  color_discrete_map={'Olive': 'olive','Yellow': 'yellow','Silver':'silver', 'Teal':'teal', 'Green':'green'})

fig_color.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    xaxis={'showgrid':True, 'gridcolor':'LightGray'},
    margin=dict(t=40, b=10, r=20),
    showlegend=False
)
fig_color

In [ ]:
df_season=df.groupby('Season', observed=True).size().reset_index(name='Count')
df_season

In [ ]:
fig_pie_season = px.pie(
    df_season, 
    values='Count', 
    names='Season', 
    title='Order Distribution by Season',
    hole=0.3, 
    height=350, 
    width=400, 
    color='Season',
    color_discrete_map={'Fall': 'orange', 'Spring': 'lightgreen', 'Summer': 'yellow', 'Winter': 'silver'}
)

fig_pie_season.update_traces(textposition='inside',
                             textinfo='label+value+percent')
fig_pie_season.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    showlegend=False,
    margin=dict(t=50, b=30),
    
)
fig_pie_season

In [ ]:
df_shipping = df.groupby('Shipping Type', observed=True).size().reset_index(name='Count')
df_shipping = df_shipping.sort_values(by='Count', ascending = False).reset_index(drop=True)
df_shipping


In [ ]:
fig_shipping = px.pie(df_shipping, values='Count', names='Shipping Type', 
                      hole=0.3, height=350, width=400, color='Shipping Type',
                      title = 'Distribution of Orders by Shipping Type'
)

fig_shipping.update_traces(textposition='inside',
                             textinfo='value+percent')
fig_shipping.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    showlegend=True,
    margin=dict(t=50, b=30),
    
)
fig_shipping

In [ ]:
df_payment = df.groupby('Payment Method', observed=True).size().reset_index(name='Count')
df_payment =df_payment.sort_values(by='Count', ascending = False).reset_index(drop=True)
df_payment

In [ ]:
fig_payment = px.bar(df_payment, x='Payment Method', y='Count', height=350, width=600, text='Count', color='Payment Method',
                    title='Number of Orders by Payment Method')

fig_payment.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    yaxis={'showgrid':True, 'gridcolor':'LightGray'},
    margin=dict(t=40, b=10, r=20),
    showlegend=False
)

fig_payment

In [ ]:
df_purchase_frequency = df.groupby(['Frequency of Purchases'], observed=True).size().reset_index(name='Count')
df_purchase_frequency = df_purchase_frequency.sort_values(by='Count', ascending = False).reset_index(drop=True)


print(df_purchase_frequency)

In [ ]:
fig_purchase_frequency = px.bar(df_purchase_frequency, 
                                x='Frequency of Purchases', 
                                y='Count', 
                                title='Order Count Per Frequency of Purchases',
                               text='Count', height=300, width=700)


fig_purchase_frequency.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    yaxis={'showgrid':True, 'gridcolor':'LightGray'},
    margin=dict(t=40, b=10, r=20)
)
fig_purchase_frequency

In [ ]:
bins = [1, 10, 20, 30, 40, 51]
labels = ['1-10', '11-20', '21-30', '31-40', '41-50']
df['Previous Purchases Group'] = pd.cut(df['Previous Purchases'], bins=bins, labels=labels, right=False)

In [ ]:
df.head()

In [ ]:
df_previous_purchases_bin = df.groupby('Previous Purchases Group', observed=True).size().reset_index(name='Count')
df_previous_purchases_bin

In [ ]:
fig_previous_purchases = px.bar(df_previous_purchases_bin, 
                                x='Previous Purchases Group', 
                                y='Count', 
                                title='Customer Distribution by Previous Purchases',
                                text='Count', height=300, width=700)

fig_previous_purchases.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    yaxis={'showgrid':True, 'gridcolor':'LightGray'},
    margin=dict(t=40, b=10, r=20)
)
fig_previous_purchases

In [ ]:
df_frequency_subscription = df.groupby(['Frequency of Purchases','Subscription Status'], observed=True).size().reset_index(name='Count')
df_frequency_subscription = df_frequency_subscription.sort_values(by='Count', ascending=False).reset_index(drop=True)

total_frequency_subscription = df.groupby('Frequency of Purchases', observed=True).size().reset_index(name='Count')
total_frequency_subscription = total_frequency_subscription.sort_values(by='Count', ascending=False).reset_index(drop=True)


print(df_frequency_subscription)
print(total_frequency_subscription)

In [ ]:
fig_frequency_subscription = px.bar(df_frequency_subscription, 
                                    x='Frequency of Purchases', 
                                    y='Count', 
                                    title='Order Distribution Across Frequency of Purchases and Subscription Status',
                                    color='Subscription Status', 
                                    text='Count',height=300)

for i, row in total_frequency_subscription.iterrows():
    fig_frequency_subscription.add_annotation(
        x=row['Frequency of Purchases'],
        y=row['Count'],
        text=str(row['Count']),
        showarrow=False,
        font=dict(size=12, color='black'),
        xanchor='auto',
        yanchor='bottom'
    )

fig_frequency_subscription.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    yaxis={'showgrid':True, 'gridcolor':'LightGray'},
    legend=dict(
        orientation='h',  # Horizontal
        yanchor='bottom', 
        y=-0.4,  # Position below the plot
        xanchor='center',
        x=0.5  # Centered horizontally
    ),
    margin=dict(t=40, b=10, r=20)
)
fig_frequency_subscription

In [ ]:
df_payment_summary = df.groupby(['Payment Method','Preferred Payment Method']).size().reset_index(name='Count')
df_payment_summary

In [ ]:
import plotly.express as px

# Create the heatmap
fig_payment_heatmap = px.density_heatmap(
    df_payment_summary,
    x='Payment Method',
    y='Preferred Payment Method',
    z='Count',  # The count column for intensity
    color_continuous_scale='Viridis',  # Choose a color scale
    title='Heatmap of Payment Method vs Preferred Payment Method',
    text_auto=True,
    width=700, height=300
)

# Display the heatmap
fig_payment_heatmap.show()


In [ ]:
df_promo_summary = df.groupby(['Discount Applied', 'Promo Code Used']).size().reset_index(name='Count')
df_promo_summary

In [ ]:
fig_promo_pie=px.pie(df_promo_summary, 
                     values='Count', names='Discount Applied',
                     color='Discount Applied',
                     title='Distribution of Orders by Discount Status',
                     hole=0.3,height=300, width=400)
fig_promo_pie


fig_promo_pie.update_traces(textposition='inside',
                             textinfo='value+percent')
fig_promo_pie.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    showlegend=True,
    margin=dict(t=50, b=30),
)

fig_promo_pie

In [ ]:
bins = [0,1, 2, 3, 4, 5.1]
labels = ['0 - 1.0', '1.0 - 2.0', '2.0 - 3.0', '3.0 - 4.0', '4.0 - 5.0']
df['Review Rating Group'] = pd.cut(df['Review Rating'], bins=bins, labels=labels, right=False)
df.head()

In [ ]:
df_review_rating_group=df.groupby(['Review Rating Group'], observed=True).size().reset_index(name='Count')
df_review_rating_group

In [ ]:
fig_review_rating=px.bar(df_review_rating_group, 
                         x='Review Rating Group', 
                         y='Count', 
                         title='Distribution of Reviews Across Rating Groups', 
                         text='Count',
                        height=300, width=700)

fig_review_rating.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    yaxis={'showgrid':True, 'gridcolor':'LightGray'},
    margin=dict(t=40, b=10, r=20)
)
fig_review_rating

In [ ]:
df_review_category=df.groupby('Category')['Review Rating'].mean().reset_index()
df_review_category.columns = ['Category','Average Review Rating']
df_review_category['Average Review Rating']=df_review_category['Average Review Rating'].round(2)
df_review_category = df_review_category.sort_values(by='Average Review Rating', ascending=False).reset_index(drop=True)
df_review_category

In [ ]:
fig_avg_review_rating = px.bar(df_review_category, 
                               x='Category',
                               y='Average Review Rating', 
                               text='Average Review Rating', 
                               title='Average Review Rating by Category',
                               color='Category',
                              height=300, width=700)
fig_avg_review_rating.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    yaxis={'showgrid':True, 'gridcolor':'LightGray'},
    margin=dict(t=40, b=10, r=20)
)

fig_avg_review_rating

In [ ]:
df_review_items=df.groupby('Item Purchased')['Review Rating'].mean().reset_index()
df_review_items.columns = ['Item Purchased','Average Review Rating']
df_review_items['Average Review Rating']=df_review_items['Average Review Rating'].round(2)
df_review_items = df_review_items.sort_values(by='Average Review Rating', ascending=False).reset_index(drop=True)
df_review_items

In [ ]:
fig_avg_review_rating_item = px.bar(df_review_items, 
                               x='Average Review Rating',
                               y='Item Purchased', 
                               text='Average Review Rating', 
                               title='Average Review Rating by Item Purchased',
                               color='Item Purchased',
                              height=700, width=650,
                                   orientation='h')
fig_avg_review_rating_item.update_layout(
    paper_bgcolor='white',  # Transparent background
    plot_bgcolor='white',  # Transparent plot area
    xaxis={'showgrid':True, 'gridcolor':'LightGray'},
    margin=dict(t=40, b=10, r=20),
    showlegend=False
)

fig_avg_review_rating_item